In [ ]:
import folium
import pandas as pd
from shapely.geometry.polygon import Polygon, Point
import ast
df=pd.read_excel(r'C:\Users\svnduw\Desktop\Bekbol\Chingis\франшиза\all_stores.xlsx')
all_board=pd.read_excel(r'C:\Users\svnduw\Desktop\Bekbol\Chingis\general_data\boarding_cities.xlsx')
all_board['polygon'] = all_board['polygon'].apply(ast.literal_eval)
df.dropna(subset=['lon', 'lat'],inplace=True)

In [ ]:
need_cities=['Aktau','Almaty','Atyrau','Karagandy','Kokshetau','Kostanay','Nur-Sultan','Pavlodar','Petropavlovsk','Semey','Taraz','Oral','Oskemen','Shymkent']
indexes=all_board[all_board.region_name.isin(need_cities)]
result={}
for index,row in indexes.iterrows():
    result[row['region_name']]=index


In [ ]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import numpy as np
for key,value in result.items():
    temp=df.copy()    
    temp['check']=None
    lat=[]
    lng=[]
    for i in range(len(all_board['polygon'][int(value)])):
        lat.append(all_board['polygon'][int(value)][i][0])
        lng.append(all_board['polygon'][int(value)][i][1])
    lons_lats_vect = np.column_stack((lat, lng)) # Reshape coordinates
    polygon = Polygon(lons_lats_vect)
    for index,row in temp.iterrows():
        point = Point(row['lon'],row['lat']) # create point
        if polygon.contains(point)==True:
            temp.at[index,'check']=True
        else:
            temp.at[index,'check']=False
    temp=temp[temp['check']==True]
    temp.to_excel(f'C:\\Users\\svnduw\\Desktop\\Bekbol\\Chingis\\франшиза\\all_cities\\{key}.xlsx',index=False)
    
    

In [ ]:
lat=[]
lng=[]
points=[]
for i in range(len(all_board['polygon'][57])):
    lat.append(all_board['polygon'][57][i][0])
    lng.append(all_board['polygon'][57][i][1])
for j in range(len(lat)):
    points.append(tuple([lng[j],lat[j]]))    

In [ ]:
# from shapely.geometry import Point
# from shapely.geometry.polygon import Polygon
# import numpy as np
# df['check']=None
# lons_lats_vect = np.column_stack((lat, lng)) # Reshape coordinates
# polygon = Polygon(lons_lats_vect)
# for index,row in df.iterrows():
#     point = Point(row['lon'],row['lat']) # create point
#     if polygon.contains(point)==True:
#         df.at[index,'check']=True
#     else:
#         df.at[index,'check']=False 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import seaborn as sns; sns.set()

In [ ]:
import os
path=r'C:\Users\svnduw\Desktop\Bekbol\Chingis\франшиза\all_cities'
files=os.listdir(path)
df=pd.read_excel(f'{path}\\{files[6]}')
X=df.loc[:,['id','lat','lon']]

In [ ]:
kmeans = KMeans(n_clusters = 6,random_state = 1)
kmeans.fit(X[X.columns[1:3]]) # Compute k-means clustering.
X['cluster_label_first'] = kmeans.fit_predict(X[X.columns[1:3]])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = kmeans.predict(X[X.columns[1:3]]) # Labels of each point
temp=df.merge(X,on=['id','lat','lon'],how='left')

In [ ]:
temp.to_excel(r'C:\Users\svnduw\Desktop\Bekbol\Chingis\франшиза\result\astana.xlsx',index=False)

In [ ]:
name='shymkent'
b=pd.DataFrame()
b.to_excel(f'C:\\Users\\svnduw\\Desktop\\Bekbol\\Chingis\\франшиза\\result\\{name}.xlsx')
for i in temp.cluster_label_first.unique().tolist():    
    a=temp[temp.cluster_label_first.isin([i])]
    with pd.ExcelWriter(f'C:\\Users\\svnduw\\Desktop\\Bekbol\\Chingis\\франшиза\\result\\{name}.xlsx',engine="openpyxl", mode='a') as writer:
        a.to_excel(writer, sheet_name=str(i),index=False)
        writer.save()
        
    


In [ ]:
temp=pd.read_excel(r'C:\Users\svnduw\Downloads\Shymkent_balanced.xlsx')

In [ ]:
temp.cluster_label_first.value_counts()

In [ ]:
zero=temp[temp.cluster_label_first==0]
one=temp[temp.cluster_label_first==1]
two=temp[temp.cluster_label_first==2]
three=temp[temp.cluster_label_first==3]
# four=temp[temp.cluster_label_first==4]
# five=temp[temp.cluster_label_first==5]
# five.drop(2112,inplace=True)


In [ ]:
print('1st franchise:',len(zero))
print('2nd franchise:',len(one))
# print('3th franchise:',len(two))


In [ ]:
m = folium.Map(location=[42.3417,69.5901], tiles="open street map", zoom_start=11,zoom_end=20)
folium.PolyLine(locations=points, color='#000000', weight=2.5).add_to(m)
for index,row in zero.iterrows():
    folium.Circle([row['lat'], row['lon']],                    
                  popup=row['id'],
                  color='#FF0000' #красный          
                   ).add_to(m)
for index,row in one.iterrows():
    folium.Circle([row['lat'], row['lon']],                    
                  popup=row['id'],
                  color='#6C3483 '  #фиолетовый
                   ).add_to(m)
for index,row in two.iterrows():
    folium.Circle([row['lat'], row['lon']],                    
                  popup=row['id'],
                  color='#28B463'   #зеленый 
                   ).add_to(m)
for index,row in three.iterrows():
    folium.Circle([row['lat'], row['lon']],                    
                  popup=row['id'],
                  color='#DEA219'
                   ).add_to(m)     #желтый яркий
    
# for index,row in four.iterrows():
#     folium.Circle([row['lat'], row['lon']],                    
#                   popup=row['id'],
#                   color='#252aa3'
#                    ).add_to(m)
# for index,row in five.iterrows():
#     folium.Circle([row['lat'], row['lon']],                    
#                   popup=row['id'],
#                   color='#138484'
#                    ).add_to(m)

In [ ]:
m

In [ ]:
m.save(outfile=r"C:\Users\svnduw\Desktop\Bekbol\Chingis\франшиза\result\shymkent.html")